In [1]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import matplotlib
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
from scipy import signal
plt.ioff()
import psutil
import gc
pd.io.parquet.get_engine('auto').__class__
%matplotlib inline
matplotlib.use('agg')

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Init Plugin
Init Graph Optimizer
Init Kernel
Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


Globals

In [2]:
cwd = os.getcwd()
print(cwd)
FREQ = 256
database_path = 'Dataset/CHB-MIT/chb-mit-scalp-eeg-database-1.0.0/'
filtered_database_path = 'Dataset/CHB-MIT/Filtered-chb-mit/'
filted_db_parquet_path = "Dataset/CHB-MIT/dataframe-parquet"
win_chb01_path = "Dataset/win_chb_01/"
external_hardisk_drive_path = '/Volumes/LaCie/Database/'
edf_file_type = ".edf"
compressed_file_type = ".csv"

#internal
files = ["../Dataset/CHB-MIT/filtered_df_csv/" + f for f in os.listdir( "../Dataset/CHB-MIT/filtered_df_csv/") if f.endswith(compressed_file_type)]
internal_window_path = "/Users/niklashjort/Desktop/Notes/Speciale/projects/MIT_Model/test_images/"

# #external
# os.chdir("/volumes/")
# files = [external_hardisk_drive_path + "filtered_df_csv/" + f for f in os.listdir(external_hardisk_drive_path + "filtered_df_csv/") if f.endswith(compressed_file_type)]

test_patient = "chb02"

files = [f for f in files if test_patient in f]


/Users/niklashjort/Desktop/Notes/Speciale/projects/DataHandling


In [3]:
for f in files[0:1]:
    print(f)

../Dataset/CHB-MIT/filtered_df_csv/chb02_16+.csv


In [4]:
def remove_cols(dataframe, col_start = 0, col_end = 0):
    if col_end == 0:
        col_end = len(dataframe.columns) - 1
    
    dataframe = dataframe.iloc[: , col_start: col_end]
    return dataframe

In [5]:
df = pd.read_csv(files[0])

In [6]:
df['class'].value_counts()

Interictal    670464
Preictal      230400
seizure        20736
Name: class, dtype: int64

In [7]:
df.columns

Index(['Unnamed: 0', 'class', 'timestamp', 'FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1',
       'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4',
       'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7',
       'T7-FT9', 'FT9-FT10', 'FT10-T8', 'STI 014'],
      dtype='object')

In [8]:
matches = ['.-0','.-1', '.-2', '.-3', '.-4', 'STI 014', 'Unnamed: 0']
selected_channels = ['F3-C3', 'T8-P8', 'FP2-F4']

def read_compressed_df(path, channel="F3-C3"):
    df = pd.read_csv(path)
    
  
    if any(x in df.columns for x in matches):
        for col_name in matches:
            try:
                df.drop(columns=col_name, inplace=True)
                
            except:
                pass

    #df = df[['timestamp', 'class', channel]]

    sz_df = df.loc[df['class'] == "seizure"].reset_index(drop=True)
    sz_df = remove_cols(sz_df)

    prei_one_df = df.loc[df['class'] == "Preictal"]
    prei_one_df = remove_cols(prei_one_df)

    inter_df = df.loc[df['class'] == "Interictal"]
    inter_df = remove_cols(inter_df)

    channels = [item for item in list(sz_df.columns) if item != "class" if item != "timestamp" if item in selected_channels]

    return (sz_df, prei_one_df, inter_df, channels)

In [9]:
def get_window(channel, start_index, data, size = 10, overlap = 0, is_sezure = False, frequency = 256):
    if(is_sezure):
        overlap = 9
    else:
        overlap = overlap
    
    start = start_index * (size - overlap) * frequency
    end = start + (size * frequency)

    date_timestamp = ""

    try:
        date_timestamp = datetime.fromtimestamp((data['timestamp'][start:start+1]/1000).tolist()[0]).strftime('%H:%M:%S')
    except:
        date_timestamp = "datetime cannot be converted"
    
    return [data[channel][start:end].tolist(), date_timestamp]

In [10]:
def get_max_window_iteration(dataframe, buffer):
    len_of_df = int(len(dataframe) / (buffer*256))
    return len_of_df


In [11]:
plt.figure(figsize=(10,10))
Fs = 256         # Define the sampling frequency,
interval = 256   # ... the interval size,
overlap = interval * 0.95  # ... and the overlap intervals
filter_order = 7
frequency_cutoff = 60
sampling_frequency = 256

import matplotlib.colors as colors

def spec_transform_save_to_folder(index, win, channel, patient_state, patient, plot_title = False):
    series = win[0]
    time_of_observation = win[1]
    try:
        series = np.array(series).astype(np.float)
    except Exception as e:
        print(f"error: {e}")
        print(f"patient_state: {patient_state} channel: {channel} index: {index} window: {series}")
    denoised_series = denoise_wavelet(series, method='BayesShrink',wavelet='db8', mode='hard',rescale_sigma=True, multichannel=False, wavelet_levels=3)
    if plot_title:
        plt.title(f"{channel} : is_seizure = {patient_state} : {time_of_observation}")
    
    b, a = signal.butter(filter_order, frequency_cutoff, btype='low', output='ba', fs=sampling_frequency)
    low_pass_signal = signal.filtfilt(b, a, denoised_series)

    

    f, t, Sxx = signal.spectrogram(low_pass_signal, fs=Fs, nperseg=interval, noverlap=overlap, nfft=interval)


    #Sxx = 10*np.log10(Sxx)
    normalize_color= colors.LogNorm(vmin=np.amin(Sxx), vmax=np.amax(Sxx))
    try:             
        # plt.pcolormesh(t, f, Sxx, norm=normalize_color,
        #         cmap='jet')
        plt.specgram(low_pass_signal, cmap='jet', Fs=256)
        #plt.ylim(0,70)
        plt.axis('off')

        #freqs, psd = signal.periodogram(denoised_series, 256, scaling='density')

        #s,f,t,im = plt.specgram(denoised_series,Fs=FREQ,cmap='jet', NFFT=int(FREQ/4), noverlap=int(FREQ/8))
        
        time_of_observation = str(time_of_observation).replace(":", "-")
        print(time_of_observation)

        if patient_state == "seizure":
            plt.savefig(f'{internal_window_path}Seizure/{patient}_{index}_{channel}_{time_of_observation}.png', bbox_inches='tight')
        elif patient_state == "interictal":
            plt.savefig(f'{internal_window_path}Interictal/{patient}_{index}_{channel}_{time_of_observation}.png', bbox_inches='tight')
        elif patient_state == "prei_one":
            plt.savefig(f'{internal_window_path}Preictal/{patient}_{index}_{channel}_{time_of_observation}.png', bbox_inches='tight')
    except:
        pass
    
    del series, denoised_series, time_of_observation, f, t, Sxx, b, a, low_pass_signal
    plt.clf()    
    plt.close()
    # Clear the current axes.
    plt.cla() 
    # Clear the current figure.
    plt.clf() 
    # Closes all the figure windows.
    plt.close('all')
    gc.collect()

In [13]:
count = 0


for filename in files:
    print("started file: " + str(filename) + " index: " + str(count))
    sz, prei_one, inter, selected_channels = read_compressed_df(filename)
    patient = re.search('../Dataset/CHB-MIT/filtered_df_csv/(.*).csv', filename).group(1)
    print(patient)
    for channel in selected_channels:
        print(f"channel: " + str(channel))
        if len(inter) > 0 and inter.empty == False:
            inter_win = [get_window(channel=channel,start_index=i, data=inter) for i in range(get_max_window_iteration(inter, 10))]
            for index, window in enumerate(inter_win):
                spec_transform_save_to_folder(win=window, index=index, channel=channel, patient_state = "interictal", patient=patient)
                del window
            del inter_win

        if len(sz) > 0 and sz.empty == False:
            sz_win = [get_window(channel=channel, start_index=i, data=sz, is_sezure=True) for i in range(get_max_window_iteration(sz, 1))]
            for index, window in enumerate(sz_win):
                spec_transform_save_to_folder(channel=channel, index=index, win=window, patient_state="seizure", patient=patient)
                del window
            del sz_win

        if len(prei_one) > 0 and prei_one.empty == False:
            prei_one_win = [get_window(channel=channel,start_index=i, data=prei_one) for i in range(get_max_window_iteration(prei_one, 10))]
            for index, window in enumerate(prei_one_win):
                spec_transform_save_to_folder(channel=channel, index=index, win=window, patient_state="prei_one", patient=patient)
                del window
            del prei_one_win

    gc.collect()
    count += 1
    print(f"memory usage = {psutil.virtual_memory().percent} : available memory = {psutil.virtual_memory().available * 100 / psutil.virtual_memory().total}")
    print(f"filename: {filename} = done : count = {count} : files left = {len(files) - count} : time of creation = {datetime.now()}")
    del sz, prei_one , inter

started file: ../Dataset/CHB-MIT/filtered_df_csv/chb02_16+.csv index: 0
chb02_16+
channel: F3-C3
10-50-23
10-50-33
10-50-43
10-50-53
10-51-03
10-51-13
10-51-23
10-51-33
10-51-43
10-51-53
10-52-03
10-52-13
10-52-23
10-52-33
10-52-43
10-52-53
10-53-03
10-53-13
10-53-23
10-53-33
10-53-43
10-53-53
10-54-03
10-54-13
10-54-23
10-54-33
10-54-43
10-54-53
10-55-03
10-55-13
10-55-23
10-55-33
10-55-43
10-55-53
10-56-03
10-56-13
10-56-23
10-56-33
10-56-43
10-56-53
10-57-03
10-57-13
10-57-23
10-57-33
10-57-43
10-57-53
10-58-03
10-58-13
10-58-23
10-58-33
10-58-43
10-58-53
10-59-03
10-59-13
10-59-23
10-59-33
10-59-43
10-59-53
11-00-03
11-00-13
11-00-23
11-00-33
11-00-43
11-00-53
11-01-03
11-01-13
11-01-23
11-01-33
11-01-43
11-01-53
11-02-03
11-02-13
11-02-23
11-02-33
11-02-43
11-02-53
11-03-03
11-03-13
11-03-23
11-03-33
11-03-43
11-03-53
11-04-03
11-04-13
11-04-23
11-04-33
11-04-43
11-04-53
11-05-03
11-05-13
11-05-23
11-05-33
11-05-43
11-05-53
11-06-03
11-06-13
11-06-23
11-06-33
11-06-43
11-06-53
11-

/var/folders/0v/m6wt8rqj7s1dcljdyjrdfxmw0000gn/T/ipykernel_58489/1730271613.py:36: UserWarning: Only one segment is calculated since parameter NFFT (=256) >= signal length (=256).
  plt.specgram(low_pass_signal, cmap='jet', Fs=256)


11-41-15
11-24-55
11-25-05
11-25-15
11-25-25
11-25-35
11-25-45
11-25-55
11-26-05
11-26-15
11-26-25
11-26-35
11-26-45
11-26-55
11-27-05
11-27-15
11-27-25
11-27-35
11-27-45
11-27-55
11-28-05
11-28-15
11-28-25
11-28-35
11-28-45
11-28-55
11-29-05
11-29-15
11-29-25
11-29-35
11-29-45
11-29-55
11-30-05
11-30-15
11-30-25
11-30-35
11-30-45
11-30-55
11-31-05
11-31-15
11-31-25
11-31-35
11-31-45
11-31-55
11-32-05
11-32-15
11-32-25
11-32-35
11-32-45
11-32-55
11-33-05
11-33-15
11-33-25
11-33-35
11-33-45
11-33-55
11-34-05
11-34-15


/usr/local/Caskroom/miniforge/base/envs/speciale_01_01/lib/python3.9/site-packages/matplotlib/axes/_axes.py:7723: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


11-34-25
11-34-35
11-34-45
11-34-55
11-35-05
11-35-15
11-35-25
11-35-35
11-35-45
11-35-55
11-36-05
11-36-15
11-36-25
11-36-35
11-36-45
11-36-55
11-37-05
11-37-15
11-37-25
11-37-35
11-37-45
11-37-55
11-38-05
11-38-15
11-38-25
11-38-35
11-38-45
11-38-55
11-39-05
11-39-15
11-39-25
11-39-35
11-39-45
channel: FP2-F4
10-50-23
10-50-33
10-50-43
10-50-53
10-51-03
10-51-13
10-51-23
10-51-33
10-51-43
10-51-53
10-52-03
10-52-13
10-52-23
10-52-33
10-52-43
10-52-53
10-53-03
10-53-13
10-53-23
10-53-33
10-53-43
10-53-53
10-54-03
10-54-13
10-54-23
10-54-33
10-54-43
10-54-53
10-55-03
10-55-13
10-55-23
10-55-33
10-55-43
10-55-53
10-56-03
10-56-13
10-56-23
10-56-33
10-56-43
10-56-53
10-57-03
10-57-13
10-57-23
10-57-33
10-57-43
10-57-53
10-58-03
10-58-13
10-58-23
10-58-33
10-58-43
10-58-53
10-59-03
10-59-13
10-59-23
10-59-33
10-59-43
10-59-53
11-00-03
11-00-13
11-00-23
11-00-33
11-00-43
11-00-53
11-01-03
11-01-13
11-01-23
11-01-33
11-01-43
11-01-53
11-02-03
11-02-13
11-02-23
11-02-33
11-02-43
11-02-53
11-